In [ ]:
#Her importerer vi modulene som trengs til dette
import json, requests

#Denne printer bare litt penere...
from pprint import pprint

In [ ]:
#Forbered etterspørsel
#Her er brukt som eksempel "Gruppering av årstrinn i grunnskolen 2014-11 - variant av Årstrinn i grunnskolen 2014-11"

url = 'http://data.ssb.no/api/klass/v1/variants/798/'
headers = {'Accept': 'application/json'}

#Gjør etterspørsel
r = requests.get(url, headers = headers)

#Av en eller annen grunn returnerer KLASS APIen json som bytestring...
#La oss decode...
cont = r.content.decode('utf8').replace("'", '"')

#Konverter til en liste vi kan bruke
cont = json.loads(cont)

#Print ut resultat
#pprint(cont)

In [ ]:
#Særbehandling ift. innhold i eksempelrequest
#Om du henter annen data fra Klass så må dette sikkert skrives om

#Tittel og info
print(cont['name'])
print('Eies av: %s' % (cont['owningSection']) )
print('Kontaktperson: %s\nEmail: %s\n' % (cont['contactPerson']['name'], cont['contactPerson']['email']))

#Loop gjennom alle klassifikasjoner og print elementer med innhold underveis
for item in cont['classificationItems']:
    code = item['code']
    parentCode = item['parentCode']
    level = item['level']
    name = item['name']
    print( f'Kode: {code}')
    
    #Av og til var denne tom, så da printer vi den ikke
    if parentCode != "":
        print( f'Foreldrekode: {parentCode}')
    
    print( f'Nivå: {level} \nNavn: {name}\n')

In [ ]:
#Ta ut til json igjen om ønskelig
s = json.dumps(cont, indent = 4 , sort_keys=True)
print(s)